In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
from pprint import pprint
import pandas as pd

In [ ]:
regions = pd.read_pickle('regions_carwale.pkl')

all_region_urls = list(regions.RegionURL)
print(all_region_urls)


In [ ]:
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(service = ChromeService(
    ChromeDriverManager().install()), options = options)

In [ ]:
carLinks   = []

for url in all_region_urls: 
    retryCount = 3
    options = webdriver.ChromeOptions()
    options.headless = True
    driver = webdriver.Chrome(service = ChromeService(
        ChromeDriverManager().install()), options = options)
    print(f"fetching data from {url=}")
    driver.get(url+'page-51/')
    print('waiting 2 seconds for browser to finish loading')
    time.sleep(2)
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True: 
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        print('waiting 2 seconds for car details to load after scroll')
        time.sleep(5)
        new_height = driver.execute_script('return document.body.scrollHeight')
        print(f"{new_height=} and {last_height=}")
        if new_height == last_height:
            print(f"{retryCount=}")
            retryCount=retryCount-1
            if retryCount<1: 
                break
        last_height = new_height

    time.sleep(5)
    cars = driver.find_elements(By.CSS_SELECTOR, 'a.o-brXWGL')
    print(f"No. of Cars: {len(cars)}")
    for car in cars: 
        carLinks.append(car.get_attribute('href'))
    driver.quit()
    print('waiting 10 seconds before next region starts')
    time.sleep(10)

In [ ]:
print(len(carLinks))

In [ ]:
cars_carwale = pd.DataFrame({'carURLs':carLinks})
cars_carwale.to_pickle('cars_carwale6.pkl')

In [ ]:
cars_carwale.shape

In [ ]:
pprint(cars_carwale)